In [3]:
!pip install transformers

     |████████████████████████████████| 1.3MB 2.7MB/s 
     |████████████████████████████████| 2.9MB 16.0MB/s 
     |████████████████████████████████| 1.1MB 34.4MB/s 
     |████████████████████████████████| 890kB 42.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=eb45fa9a548d66ce191f97df093805aa7413a24c691d63ba2ee01e48daf96ddd
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import torch
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration
from IPython.display import display, Markdown

torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
torch.cuda.is_available()

False

In [6]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

In [10]:
MS_NEWS = """
 Morgan Stanley today announced a donation of $2.5 million to hunger relief organizations worldwide, equaling 25 million meals, as a result of its latest employee campaign: Move for 
 Meals. From October 16-18, nearly 10,500 employees in more than 440 cities around the world participated in the Move for Meals Campaign, walking 250 million steps to raise awareness 
 about global hunger in recognition of World Food Day. 

The Morgan Stanley Move for Meals Campaign builds on the Firm’s previously announced COVID-19 Hunger Relief Campaign, which took place at the start of the pandemic and raised over 
$2 million globally in employee donations alone. Since then, the needs of our communities have unfortunately worsened, with one in four people malnourished globally and, in the U.S. 
alone, one in six people possibly facing hunger in 2020 as a result of the pandemic.

“The high level of employee engagement, combined with the Firm’s monetary commitment, is a true testament to our employees' dedication to giving back and fighting hunger,” said Joan 
Steinberg, Global Head of Philanthropy at Morgan Stanley. “Food insecurity has always been a crucial issue that millions are faced with, but now with the issue exacerbated by the 
pandemic, it is important that we leverage our global reach to support over 20 partner organizations worldwide that are on the front lines of addressing hunger disparities during 
the current health crisis.”

Since the start of the pandemic, food banks across the U.S. and around the world have seen both their food supplies drop and their demand increase. Feeding America, a long-time 
charitable partner of Morgan Stanley that fights hunger by supporting 200 network member food banks across in the U.S., reports their need has nearly doubled since the pandemic hit 
the U.S. According to their predictions, nearly 54 million Americans could need food assistance this year ‒ while food donations have fallen by 55 percent.

Through the Move for Meals and COVID-19 Hunger Relief Campaigns, Morgan Stanley has supported critical feeding partners in all of our major locations globally. In total, this brings 
the Firm’s commitment to support COVID-19 relief efforts to $80 million, including prior grants to food assistance partners, such as Feeding America and Akshaya Patra.

To learn more about Morgan Stanley’s giving back efforts, please see here.

Morgan Stanley (NYSE: MS) is a leading global financial services firm providing investment banking, securities, wealth management and investment management services. With offices 
in more than 41 countries, the Firm's employees serve clients worldwide including corporations, governments, institutions and individuals. For more information about Morgan Stanley, 
please visit www.morganstanley.com. 
""".replace('\n','')

In [11]:
article_input_ids = tokenizer.batch_encode_plus([MS_NEWS], return_tensors='pt', max_length=1024)['input_ids'].to(torch_device)
summary_ids = model.generate(article_input_ids,
                             num_beams=4,
                             length_penalty=2.0,
                             max_length=142,
                             min_len=56,
                             no_repeat_ngram_size=3)

In [12]:
summary_txt = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
display(Markdown('> **Summary: **'+summary_txt))

> **Summary: **From October 16-18, nearly 10,500 employees in more than 440 cities around the world participated in the Move for Meals Campaign. Morgan Stanley’s commitment to support COVID-19 relief efforts to $80 million, including prior grants to food assistance partners, such as Feeding America and Akshaya Patra.

In [30]:
article_input_ids = tokenizer.batch_encode_plus([MS_NEWS], return_tensors='pt', max_length=1024)['input_ids'].to(torch_device)
summary_ids = model.generate(article_input_ids,
                             num_beams=4,
                             min_length=200,
                             max_length=500, 
                             no_repeat_ngram_size=3)

summary_txt = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
display(Markdown('> **Summary: **'+summary_txt))

> **Summary: **From October 16-18, nearly 10,500 employees in more than 440 cities around the world participated in the Move for Meals Campaign. Morgan Stanley’s commitment to support COVID-19 relief efforts to $80 million, including prior grants to food assistance partners, such as Feeding America and Akshaya Patra. In the U.S. alone, nearly 54 million Americans could need food assistance this year ‒ while food donations have fallen by 55 percent.“Food insecurity has always been a crucial issue that millions are faced with, but now with the issue exacerbated by the pandemic, it is important that we leverage our global reach to support over 20 partner organizations worldwide that are on the front lines of addressing hunger disparities,” said Joan Steinberg, Global Head of Philanthropy at Morgan Stanley. “The high level of employee engagement... is a true testament to our employees' dedication to giving back and fighting hunger,’ said Steinberg.